In [19]:
import pandas as pd
from tree_sitter import Language

import sys
sys.path.append('function_parser/function_parser/')



In [21]:
from language_data import LANGUAGE_METADATA
from process import DataProcessor
import os
import time
from tree_sitter import Language, Parser

Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-c'  
  ]
)

False

In [36]:
from tree_sitter import Language, Parser

Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages-cpp.so',

  # Include one or more languages
  [
    'tree-sitter-cpp'  
  ]
)

True

In [22]:
C_LANGUAGE = Language('build/my-languages.so', 'c')

In [39]:
CPP_LANGUAGE = Language('build/my-languages-cpp.so', 'cpp')

In [40]:
cpp_parser = Parser()
cpp_parser.set_language(CPP_LANGUAGE)

In [240]:
c_parser = Parser()
c_parser.set_language(C_LANGUAGE)

In [241]:
def walk(dir_name: str, ext: str):
    return [os.path.join(dp, f) for dp, dn, filenames in os.walk(dir_name) 
            for f in filenames if os.path.splitext(f)[1] == '.' + ext]

In [26]:
files = walk('repos/openssl', 'c')

In [27]:
demo_file_name = files[3]
with open(demo_file_name) as source_code:
    blob = source_code.read()
lines = blob.splitlines(True)

In [28]:
tree = parser.parse(bytes(blob, "utf-8"))

In [35]:
t = time.time()
for demo_file_name in files:
    with open(demo_file_name) as source_code:
        blob = source_code.read()
    lines = blob.splitlines(True)
    tree = parser.parse(bytes(blob, "utf-8"))
    #classes = (node for node in tree.root_node.children if node.type == "class_declaration")
    definitions = []
    for idx, node in enumerate(tree.root_node.children):
        if node.type == "function_definition":
            start_line = -1
            end_line = -1
            for idx_c, node_c in enumerate(node.children):
                if start_line == -1 or node_c.start_point[0] < start_line:
                    start_line = node_c.start_point[0]
                if end_line < node_c.end_point[0]:
                    end_line = node_c.end_point[0]
            definitions.append({"function": ''.join(lines[start_line:end_line]), "start_line": start_line, "end_line": end_line})
        
print(time.time() - t)

2.985689878463745


In [43]:
files = walk('repos/faiss', 'cpp')

In [60]:
demo_file_name = files[3]
with open(demo_file_name) as source_code:
    blob = source_code.read()
lines = blob.splitlines(True)

In [61]:
cpp_tree = cpp_parser.parse(bytes(blob, "utf-8"))

In [238]:
func_nodes = []
FUNC_NODE_TYPES = ['function_definition', 'preproc_function_def']
for node in cpp_tree.root_node.children:
    if node.type in FUNC_NODE_TYPES:
        func_nodes.append(node)
    for node_1 in node.children:
        if node_1.type in FUNC_NODE_TYPES:
            func_nodes.append(node_1)
        for node_2 in node_1.children:
            if node_2.type in FUNC_NODE_TYPES:
                func_nodes.append(node_2)
            for node_3 in node_2.children:
                if node_3.type in FUNC_NODE_TYPES:
                    func_nodes.append(node_3)
                for node_4 in node_3.children:
                    if node_4.type in FUNC_NODE_TYPES:
                        func_nodes.append(node_4)
                    for node_5 in node_4.children:
                        if node_5.type in FUNC_NODE_TYPES:
                            func_nodes.append(node_5)
                        for node_6 in node_5.children:
                            if node_6.type in FUNC_NODE_TYPES:
                                func_nodes.append(node_6)
                            for node_7 in node_6.children:
                                if node_7.type in FUNC_NODE_TYPES:
                                    func_nodes.append(node_7)

In [242]:
definitions = []
for node in func_nodes:
    start_line = node.start_point[0]
    end_line = node.end_point[0]
    definitions.append({"function": ''.join(lines[start_line:end_line]), "start_line": start_line, "end_line": end_line})   

In [243]:
import pandas as pd

In [244]:
df = pd.read_csv('out/diffs.csv')

In [247]:
df.head(10)

,repo_url,commit,cve,filename,before_lines,after_lines,before_gz_b64,after_gz_b64
0,https://android.googlesource.com/device/google...,7df7ec13b1d222ac3a66797fbe432605ea8f973f,CVE-2017-0812,audio/hal/audio_hw.c,1,1,H4sIAC0M/mMC/+y9CXPbSLK2+1fQnvCE7bYtVhVWz9gn1B...,H4sIAC0M/mMC/+y9CXPbSLK2+1fQnvCE7fbCKhQ2z9gn1B...
1,https://android.googlesource.com/kernel/common,69bfe2d957d903521d32324190c2754cb073be15,CVE-2015-6640,kernel/sys.c,1,1,H4sIAH8S/mMC/+y9/3LbRtYt+rffouPUpOyMLBNA41cyyR...,H4sIAH8S/mMC/+y9/3LbRtYt+rffouPUpOyMLLOBxq9kkj...
2,https://android.googlesource.com/platform/boot...,28a566f7731b4cb76d2a9ba16d997ac5aeb07dad,CVE-2016-0849,minzip/SysUtil.c,1,1,H4sIAP4R/mMC/+1djVvbRtL/VzbuSwsEo++vBrtHEpLmXh...,H4sIAP4R/mMC/+1dDXvTuJb+KyKznSnQJP7+gCazBQrDbq...
3,https://android.googlesource.com/platform/dalvik,338aeaf28e9981c15d0673b18487dba61eb5447c,CVE-2016-3758,libdex/OptInvocation.cpp,1,1,H4sIAEsL/mMC/+1dC1fbuLb+K2rmzgxM8/I7aSFzKNCWng...,H4sIAEsL/mMC/+1dC1vbONb+K2pmZwamufnutJBZCrRltg...
4,https://android.googlesource.com/platform/exte...,5d4405f601fa11a8955fd7611532c982420e4206,CVE-2016-2428,libAACdec/src/aacdec_drc.cpp,1,1,H4sIAJwK/mMC/+29jXrbyLUt+CplZybH7rYkFv7RHftcSq...,H4sIAJwK/mMC/+29j17bSLYu+ioV5t7ZSSeA9V/qbrKPMU...
5,https://android.googlesource.com/platform/exte...,8e3be529372892e20ccf196809bc73276c822189,CVE-2017-13188,libSBRdec/src/lpp_tran.cpp,1,1,H4sIAJ4R/mMC/+29jXrbSJIt+CpZ7t0eqWxJxD9QVXavLE...,H4sIAJ4R/mMC/+29jVrbWLY2eCu70zN9QiWA9S9VVdIfIV...
6,https://android.googlesource.com/platform/exte...,0360aa7c418152a3e5e335a065ac3629cbb09559,CVE-2014-7914,btif/src/btif_dm.c,1,1,H4sIAKsV/mMC/+y9i3faSLY9/K+oM6tnJd15oLeUnvRdGH...,H4sIAKsV/mMC/+y9i3fiRrY9/K8oPSuzupN+oLfUmc5dGH...
7,https://android.googlesource.com/platform/exte...,c677ee92595335233eb0e7b59809a1a94e7a678a,CVE-2016-0850,include/bt_target.h,1,1,H4sIAKES/mMC/+y9i3LbSJat/So57jgdtse2iDvQ1VUTFA...,H4sIAKES/mMC/+y9i3LbSJat/So57jgdtse2iDvQ1VUTFA...
8,https://android.googlesource.com/platform/exte...,c677ee92595335233eb0e7b59809a1a94e7a678a,CVE-2016-0850,stack/btm/btm_int.h,1,1,H4sIAKES/mMC/+19bXubSpbt5/svmPS9PUmfJBbv0KeTfm...,H4sIAKES/mMC/+29e3ubSpYu/vf5Fkz69+tJeiexuEPvTv...
9,https://android.googlesource.com/platform/exte...,c677ee92595335233eb0e7b59809a1a94e7a678a,CVE-2016-0850,stack/btm/btm_sec.c,1,1,H4sIAKES/mMC/+y9fXfa1tYt/vfvW6jpSEfS5gW9S+lJ7i...,H4sIAKES/mMC/+y9a3fayNYt/Pn9F9XpkR5Jdy5IKt3SOz...


In [246]:
def decode_file(s):
    return gzip.decompress(base64.b64decode(s)).decode('UTF-8')

In [ ]:
df[]